### Import Libs

In [198]:
# Adding needed libraries and reading data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from databricks import koalas as ks
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Load Data

In [199]:
kdf_train =  ks.from_pandas(pd.read_csv('train.csv'))
kdf_test =  ks.from_pandas(pd.read_csv('test.csv'))
print(kdf_train.shape)
print(kdf_test.shape)

(1460, 81)
(1459, 80)


### Drop Columns

In [200]:
train_labels = kdf_train['SalePrice']
kdf_train.pop('SalePrice')
features = ks.concat([kdf_train, kdf_test])
features = features.drop(['Utilities', 'RoofMatl', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'LowQualFinSF',
               'BsmtFullBath', 'BsmtHalfBath', 'Functional', 'GarageYrBlt', 'GarageArea', 'GarageCond', 'WoodDeckSF',
               'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal'],
              axis=1)
print(features.shape)

(2919, 56)


### FillNa & Converting

In [201]:
# MSSubClass as str
features['MSSubClass'] = features['MSSubClass'].astype(str)

# MSZoning NA in pred. filling with most popular values
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

# LotFrontage  NA in all. I suppose NA means 0
features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())

# Alley  NA in all. NA means no access
features['Alley'] = features['Alley'].fillna('NOACCESS')

# Converting OverallCond to str
features.OverallCond = features.OverallCond.astype(str)

# MasVnrType NA in all. filling with most popular values
features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
# NA in all. NA means No basement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('NoBSMT')

# TotalBsmtSF  NA in pred. I suppose NA means 0
features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

# Electrical NA in pred. filling with most popular values
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

# KitchenAbvGr to categorical
features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

# KitchenQual NA in pred. filling with most popular values
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

# FireplaceQu  NA in all. NA means No Fireplace
features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

# GarageType, GarageFinish, GarageQual  NA in all. NA means No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    features[col] = features[col].fillna('NoGRG')

# GarageCars  NA in pred. I suppose NA means 0
features['GarageCars'] = features['GarageCars'].fillna(0.0)

# SaleType NA in pred. filling with most popular values
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

# Year and Month to categorical
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

# Adding total sqfootage feature and removing Basement, 1st and 2nd floor features
features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features = features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1)

### Log transformation

In [202]:
## Standardizing numeric features
numeric_features = features.loc[:,['LotFrontage', 'LotArea', 'GrLivArea', 'TotalSF']]
#numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

### Converting categorical data to dummies

In [203]:
print(features.shape)

features = ks.get_dummies(data=features, columns=["Condition1"])
print(features.shape)
print(features.columns)
features
# Getting Dummies from Condition1 and Condition2
# conditions = set([x for x in features['Condition1']] + [x for x in features['Condition2']])
# dummies = ks.DataFrame(data=np.zeros((len(features.index), len(conditions))),
#                        index=features.index, columns=conditions)
# for i, cond in enumerate(zip(features['Condition1'], features['Condition2'])):
#     dummies.ix[i, cond] = 1
# features = ks.concat([features, dummies.add_prefix('Condition_')], axis=1)
# features = features.drop(['Condition1', 'Condition2'], axis=1)

# # Getting Dummies from Exterior1st and Exterior2nd
# exteriors = set([x for x in features['Exterior1st']] + [x for x in features['Exterior2nd']])
# dummies = ks.DataFrame(data=np.zeros((len(features.index), len(exteriors))),
#                        index=features.index, columns=exteriors)
# for i, ext in enumerate(zip(features['Exterior1st'], features['Exterior2nd'])):
#     dummies.ix[i, ext] = 1
# features = ks.concat([features, dummies.add_prefix('Exterior_')], axis=1)
# features = features.drop(['Exterior1st', 'Exterior2nd', 'Exterior_nan'], axis=1)

# # Getting Dummies from all other categorical vars
# for col in features.dtypes[features.dtypes == 'object'].index:
#     for_dummy = features.pop(col)
#     features = pd.concat([features, pd.get_dummies(for_dummy, prefix=col)], axis=1)

(2919, 54)
(2919, 62)
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'HeatingQC', 'CentralAir', 'Electrical', 'GrLivArea', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageCars', 'GarageQual', 'PavedDrive', 'MoSold',
       'YrSold', 'SaleType', 'SaleCondition', 'TotalSF', 'Condition1_Artery',
       'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA',
       'Condition1_PosN', 'Condition1_RRAe', 'Condition1_RRAn',
       'Condition1_RRNe', 'Conditio

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,Electrical,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,PavedDrive,MoSold,YrSold,SaleType,SaleCondition,TotalSF,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn
0,1,60,RL,65.000000,8450,Pave,NOACCESS,Reg,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,2Story,7,5,2003,2003,Gable,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,Ex,Y,SBrkr,1710,2,1,3,1,Gd,8,0,NoFP,Attchd,RFn,2.0,TA,Y,2,2008,WD,Normal,2566.0,0,0,1,0,0,0,0,0,0
1,2,20,RL,80.000000,9600,Pave,NOACCESS,Reg,Lvl,FR2,Gtl,Veenker,Norm,1Fam,1Story,6,8,1976,1976,Gable,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,Ex,Y,SBrkr,1262,2,0,3,1,TA,6,1,TA,Attchd,RFn,2.0,TA,Y,5,2007,WD,Normal,2524.0,0,1,0,0,0,0,0,0,0
2,3,60,RL,68.000000,11250,Pave,NOACCESS,IR1,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,2Story,7,5,2001,2002,Gable,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,Ex,Y,SBrkr,1786,2,1,3,1,Gd,6,1,TA,Attchd,RFn,2.0,TA,Y,9,2008,WD,Normal,2706.0,0,0,1,0,0,0,0,0,0
3,4,70,RL,60.000000,9550,Pave,NOACCESS,IR1,Lvl,Corner,Gtl,Crawfor,Norm,1Fam,2Story,7,5,1915,1970,Gable,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,Gd,Y,SBrkr,1717,1,0,3,1,Gd,7,1,Gd,Detchd,Unf,3.0,TA,Y,2,2006,WD,Abnorml,2473.0,0,0,1,0,0,0,0,0,0
4,5,60,RL,84.000000,14260,Pave,NOACCESS,IR1,Lvl,FR2,Gtl,NoRidge,Norm,1Fam,2Story,8,5,2000,2000,Gable,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,Ex,Y,SBrkr,2198,2,1,4,1,Gd,9,1,TA,Attchd,RFn,3.0,TA,Y,12,2008,WD,Normal,3343.0,0,0,1,0,0,0,0,0,0
5,6,50,RL,85.000000,14115,Pave,NOACCESS,IR1,Lvl,Inside,Gtl,Mitchel,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,VinylSd,VinylSd,None,TA,TA,Wood,Gd,TA,No,GLQ,Unf,Ex,Y,SBrkr,1362,1,1,1,1,TA,5,0,NoFP,Attchd,Unf,2.0,TA,Y,10,2009,WD,Normal,2158.0,0,0,1,0,0,0,0,0,0
6,7,20,RL,75.000000,10084,Pave,NOACCESS,Reg,Lvl,Inside,Gtl,Somerst,Norm,1Fam,1Story,8,5,2004,2005,Gable,VinylSd,VinylSd,Stone,Gd,TA,PConc,Ex,TA,Av,GLQ,Unf,Ex,Y,SBrkr,1694,2,0,3,1,Gd,7,1,Gd,Attchd,RFn,2.0,TA,Y,8,2007,WD,Normal,3380.0,0,0,1,0,0,0,0,0,0
7,8,60,RL,69.305795,10382,Pave,NOACCESS,IR1,Lvl,Corner,Gtl,NWAmes,Norm,1Fam,2Story,7,6,1973,1973,Gable,HdBoard,HdBoard,Stone,TA,TA,CBlock,Gd,TA,Mn,ALQ,BLQ,Ex,Y,SBrkr,2090,2,1,3,1,TA,7,2,TA,Attchd,RFn,2.0,TA,Y,11,2009,WD,Normal,3197.0,0,0,0,0,1,0,0,0,0
8,9,50,RM,51.000000,6120,Pave,NOACCESS,Reg,Lvl,Inside,Gtl,OldTown,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,BrkFace,Wd Shng,None,TA,TA,BrkTil,TA,TA,No,Unf,Unf,Gd,Y,FuseF,1774,2,0,2,2,TA,8,2,TA,Detchd,Unf,2.0,Fa,Y,4,2008,WD,Abnorml,2726.0,1,0,0,0,0,0,0,0,0
9,10,190,RL,50.000000,7420,Pave,NOACCESS,Reg,Lvl,Corner,Gtl,BrkSide,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,MetalSd,MetalSd,None,TA,TA,BrkTil,TA,TA,No,GLQ,Unf,Ex,Y,SBrkr,1077,1,0,2,2,TA,5,2,TA,Attchd,RFn,1.0,Gd,Y,1,2008,WD,Normal,2068.0,1,0,0,0,0,0,0,0,0


### Plot

In [204]:
# features.plot.hist('OverallCond	')
# type(features)

### Split Train & test df

In [216]:
print(features.shape)
kdf_train = features.iloc[:1460,:]
print(type(train_labels) )
#train_labels = train_labels.to_frame()
print(type(train_labels) )
print(train_labels)
kdf_test = ks.sql('select * from {features} where Id > 1460')
kdf_train = ks.concat([train_labels,kdf_train], axis = "index")
print(type(train_labels) )
print(kdf_train.shape)
print(kdf_test.shape)



(2919, 62)
databricks.koalas.frame.DataFrame
databricks.koalas.frame.DataFrame
     SalePrice
0       208500
1       181500
2       223500
3       140000
4       250000
5       143000
6       307000
7       200000
8       129900
9       118000
10      129500
11      345000
12      144000
13      279500
14      157000
15      132000
16      149000
17       90000
18      159000
19      139000
20      325300
21      139400
22      230000
23      129900
24      154000
25      256300
26      134800
27      306000
28      207500
29       68500
30       40000
31      149350
32      179900
33      165500
34      277500
35      309000
36      145000
37      153000
38      109000
39       82000
40      160000
41      170000
42      144000
43      130250
44      141000
45      319900
46      239686
47      249700
48      113000
49      127000
50      177000
51      114500
52      110000
53      385000
54      130000
55      180500
56      172500
57      196500
58      438780
59      124900
60    

databricks.koalas.frame.DataFrame
(2920, 63)
(1459, 62)


### Merge train with lable

In [206]:
type(train_labels)
train_labels.to_frame()

type(train_labels.shape)
type(train_labels)

AttributeError: 'DataFrame' object has no attribute 'to_frame'